In [ ]:
'''
Import libraries and install openxyl library to convert excel file to CSV file and smtplip helps with handling email
notifications
'''
import os
import pandas as pd
import time
import logging
import logging.handlers
import smtplib
from email.message import EmailMessage


In [ ]:
# Get the current working directory
current_working_directory = os.getcwd()
print("Current Working Directory:", current_working_directory)

In [ ]:
# Set the paths for files
excel_file_path = 'list_fav_meals.xlsx'
timestamp_file_path = 'last_run_timestamp.txt' # no need to create a text file

In [ ]:
# Check if the file exists
if not os.path.exists(timestamp_file_path):
    # Open the file in write mode to create it
    with open(timestamp_file_path, 'w') as file:
        pass  # This does nothing, creating an empty file
    print(f"File '{timestamp_file_path}' has been created.")
else:
    print(f"File '{timestamp_file_path}' already exists.")


In [ ]:
# Extract base name of the Excel file without extension to create CSV file name
excel_file_name = os.path.splitext(os.path.basename(excel_file_path))[0]
csv_file_path = f'{excel_file_name}.csv'

# Get the current timestamp
current_timestamp = time.time()
print("Current timestamp:", current_timestamp)


In [ ]:
# Check if the timestamp file exists
if os.path.exists(timestamp_file_path):
    # Read the last run timestamp from the file
    with open(timestamp_file_path, 'r') as file:
        last_run_timestamp_str = file.read().strip()  # Remove leading/trailing whitespaces

    try:
        # Attempt to convert the read timestamp to a float
        last_run_timestamp = float(last_run_timestamp_str)
    except ValueError:
        # Handle the case where the file doesn't contain a valid float
        logging.info("Invalid timestamp in the file. Using the current timestamp.")
        last_run_timestamp = 0.0  # Set a default value or handle this case as needed
else:
    # Handle the case where the file doesn't exist
    logging.info("Timestamp file not found. Using the current timestamp.")
    print("Timestamp file not found. Using the current timestamp.")
    last_run_timestamp = 0.0  # Set a default value or handle this case as needed


In [ ]:
# Compare with the current timestamp to determine if the Excel file has been run before
if current_timestamp - last_run_timestamp < 120:  # timestamp for two minutes, and 2419200 is the timestamp calculated number for 28 days
    print("Excel file was run within the last 2 minutes. Not running it again.")
else:
    logging.info("Processing the Excel file.")
    print("Processing the Excel file.")

    # Load data from Excel file (replace this with your Excel processing logic)
    try:
        df_excel = pd.read_excel(excel_file_path)

        # Your Excel processing logic goes here

        # Save the processed data to CSV
        df_excel.to_csv(csv_file_path, index=False)
        logging.info(f"Data processed and saved to {csv_file_path}")
        print(f"Data processed and saved to {csv_file_path}")

        # Update the timestamp file with the current timestamp
        with open(timestamp_file_path, 'w') as file:
            file.write(str(current_timestamp))
            logging.info("Timestamp file updated.")
            print("Timestamp file updated.")
    except Exception as e:
        logging.info(f"Error processing Excel file: {e}")
        print(f"Error processing Excel file: {e}")

In [ ]:

# How to Handle Email Notification Instructions.
# Note: App Passwordis different than regular email password, see instructions below

# Email configuration
# How to set up an APP PASSWORD on a Google account: https://www.youtube.com/watch?v=IWxwWFTlTUQ
email_sender = 'account_testing@outlook.com'  # Sender email info. It is a new mock email, we need to replace it for final version
email_password = 'type_here_your_app_password'  # App Password created on Outlook/Google account
email_recipient = 'recipient_email@gmail.com' # Replace with your email address

# Function to send email notification and protocols
def send_email(subject, body):
    msg = EmailMessage()
    msg.set_content(body)
    msg['Subject'] = subject
    msg['From'] = email_sender
    msg['To'] = email_recipient

    try:
        server = smtplib.SMTP('smtp-mail.outlook.com', 587) #sender@outlook.com
        server.starttls()
        server.login(email_sender, email_password)
        server.send_message(msg)
        server.quit()
        print("Email notification sent successfully.")
    except Exception as e:
        print(f"Error sending email notification: {e}")


# Inside the try block where data is processed or error occurs
try:
    # Your Excel processing logic goes here
    # Save the processed data to CSV
    df_excel.to_csv(csv_file_path, index=False)
    logging.info(f"Data processed and saved to {csv_file_path}")
    print(f"Data processed and saved to {csv_file_path}")

    # Send email notification when the file is successfully processed
    send_email("Data Processed Successfully", f"Data processed and saved to {csv_file_path}")
except Exception as e:
    logging.info(f"Error processing Excel file: {e}")
    print(f"Error processing Excel file: {e}")

    # Send email notification when there is an error
    send_email("Error Processing Excel File", f"Error processing Excel file: {e}")